In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20171016_gRNA1_Pax7_Sox9/sec/pax7counts'

In [4]:
# Add experiment date here to apply to dataframe
expt_date = '20171016'
treatment = 'SMPD3gRNA1'
result_type = 'Pax7Counts'

path = os.path.abspath('')+'/pax7csv/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Parse Image name
    df['Embryo'], df['Somites'], df['Section'] = zip(*df['Image'].map(lambda x: x.split('_')))

    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Date,Image,Embryo,Somites,Section
0,CntlSide,29,1285.597,44.331,10.478,255,11304.389,20171016,Emb7_8ss_sec6,Emb7,8ss,sec6
1,ExptSide,24,1443.946,60.164,11.572,255,15341.925,20171016,Emb7_8ss_sec6,Emb7,8ss,sec6
0,CntlSide,12,1304.096,108.675,17.044,255,27712.043,20171016,Emb9_9ss_sec4,Emb9,9ss,sec4
1,ExptSide,17,1890.754,111.221,30.858,255,28361.308,20171016,Emb9_9ss_sec4,Emb9,9ss,sec4
0,CntlSide,34,1748.998,51.441,8.733,255,13117.482,20171016,Emb7_8ss_sec5,Emb7,8ss,sec5


In [6]:
# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Embryo', 'Slice'])['Count']).mean())
mean_sections
# Extract means for Cntl and Expt counts
pax7_cntl = mean_sections.xs('CntlSide', level='Slice')['Count'] 
pax7_expt = mean_sections.xs('ExptSide', level='Slice')['Count']

# Generate ratios as Expt/Cntl
pax7_ratios = pd.DataFrame(pax7_expt / pax7_cntl)
pax7_ratios.columns = ['Expt/Cntl Pax7']
pax7_ratios.head()

,Expt/Cntl Pax7
Embryo,
Emb2,0.973545
Emb3,0.798561
Emb5,0.674877
Emb6,1.125000
Emb7,0.901786


In [7]:
# Combine processed values into single dataframe and output as csv file
pax7_cntl = pd.DataFrame(pax7_cntl)
pax7_cntl.columns = ['Cntl Pax7']
pax7_expt = pd.DataFrame(pax7_expt)
pax7_expt.columns = ['Expt Pax7']

results = pd.concat([pax7_cntl, pax7_expt, pax7_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')